# GitHub repositories analysis

In [9]:
import pandas as pd
import os
import glob

## Read the data

In [28]:
files = glob.glob("./data/**/*.csv", recursive=True)
print("Encontrados", len(files), "ficheros")

Encontrados 15 ficheros


In [48]:
df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

In [37]:
df.head(3)

,Unnamed: 0,name,ts,stars_count,views_total,views_uniques,clones_total,clones_uniques
0,0,ic-gradle,2022-06-29,0,0,0,1,1
1,1,istio-wasm-filter-golang,2022-06-29,0,3,2,0,0
2,2,jwt,2022-06-29,1,0,0,2,1


## Data analysis

### Views

In [38]:
views = df.groupby(['name']).agg({"views_total": ["sum"]})
views = views[views[('views_total', 'sum')] > 0]
views.sort_values(by=[('views_total', 'sum')], ascending=False)

,views_total
,sum
name,
quarkus-kotlin-example,92
golang-rest-api-cassandra,88
kotlin-ktor,86
kustomize-vs-helm,43
ta,38
pelorus-workshop,31
library,28
spring-kotlin-crud,24


### Clones

In [39]:
clones = df.groupby(['name']).agg({"clones_uniques": ["sum"]})
clones = clones[clones[('clones_uniques', 'sum')] > 0]
clones.sort_values(by=[('clones_uniques', 'sum')], ascending=False)

,clones_uniques
,sum
name,
ta,293
golang-rest-api-cassandra,237
mqtt-golang-influxdb,119
iot-devices-crud,95
pelorus-workshop,14
b0rr3g0-blog,9
ml-github,6
service-mesh-wasm-go,5


# Stars

In [40]:
stars = df.groupby(['name']).agg({"stars_count": ["max"]})
stars = stars[stars[('stars_count', 'max')] > 0]
stars.sort_values(by=[('stars_count', 'max')], ascending=False)

,stars_count
,max
name,
mqtt-golang-influxdb,9
spring-kotlin-crud,6
kotlin-ktor,5
kops-aws,4
kustomize-vs-helm,3
ta,3
golang-k8s-helm-helloworld,2
golang-rest-api-cassandra,2
